In [1]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [2]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [3]:
def answer_one():
    
    
    return (len(spam_data[spam_data['target'] == 1])/len(spam_data))*100

In [4]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    cv = CountVectorizer()
    data = cv.fit(X_train)
    vocab = []
    for i in data.vocabulary_:
        vocab.append(i)
    vocab = np.array(vocab)
    
    return max(vocab, key = len)
    

In [6]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    vect = CountVectorizer().fit(X_train)
    X_train_transform = vect.transform(X_train)
    clf = MultinomialNB(alpha = 0.1)
    clfmnb = clf.fit(X_train_transform, y_train)
    pred = clfmnb.predict(vect.transform(X_test))
    aoc = roc_auc_score(y_test, pred)
    
    
    
    return aoc

In [8]:
answer_three()

0.97208121827411165

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    tfidf = TfidfVectorizer()
    vect = tfidf.fit(X_train)
    feature_names = np.array(vect.get_feature_names())
    X_train_vectorized = vect.transform(X_train)
    df = pd.DataFrame()
    sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()
    smallest = feature_names[sorted_tfidf_index[:20]]
    largest = feature_names[sorted_tfidf_index[-20:]]
    tfidf_values = X_train_vectorized.max(0).toarray()[0]
    small_values = sorted(tfidf_values)[:20]
    largest_values = sorted(tfidf_values)[-20:]
    small_series = pd.DataFrame(smallest)
    small_series['tfidf'] = small_values
    small_1 = small_series[small_series['tfidf'] == small_series['tfidf'][0]].sort_values(by = 0)
    small_2 = small_series[small_series['tfidf'] == small_series['tfidf'][19]].sort_values(by = 0)
    frames = [small_1, small_2]
    small_final = pd.concat(frames)
    small_final.set_index(0, inplace = True)
    large_series = pd.DataFrame(largest)
    large_series['tfidf'] = largest_values
    large_series.sort_values(by = 'tfidf', ascending = False, inplace = True)
    large_final = large_series[large_series['tfidf'] == large_series['tfidf'][10]].sort_values(by = 0).set_index(0)
    del small_final.index.name
    large_final = pd.concat([large_final, large_series[-2:]])
    ans = (small_final['tfidf'], large_final['tfidf'])
    
    
    
    return ans

In [10]:
answer_four()

(aaniye          0.074475
 athletic        0.074475
 chef            0.074475
 companion       0.074475
 courageous      0.074475
 dependable      0.074475
 determined      0.074475
 exterminator    0.074475
 healer          0.074475
 listener        0.074475
 organizer       0.074475
 pest            0.074475
 psychiatrist    0.074475
 psychologist    0.074475
 pudunga         0.074475
 stylist         0.074475
 sympathetic     0.074475
 venaam          0.074475
 diwali          0.091250
 mornings        0.091250
 Name: tfidf, dtype: float64, 146tf150p    1.000000
 645          1.000000
 anything     1.000000
 anytime      1.000000
 beerage      1.000000
 done         1.000000
 er           1.000000
 havent       1.000000
 home         1.000000
 lei          1.000000
 nite         1.000000
 ok           1.000000
 okie         1.000000
 thank        1.000000
 thanx        1.000000
 too          1.000000
 where        1.000000
 yup          1.000000
 1            0.980166
 0            

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [12]:
def answer_five():
    tfidf = TfidfVectorizer(min_df = 3)
    vect = tfidf.fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    clf = MultinomialNB(alpha = 0.1)
    clfnb = clf.fit(X_train_vectorized, y_train)
    pred = clfnb.predict(vect.transform(X_test))
    auc_score = roc_auc_score(y_test, pred)
    
    
    
    return auc_score

In [13]:
answer_five()

0.94162436548223349

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [14]:
def answer_six():
    not_spam = [i for i in spam_data['text'][spam_data['target'] == 0]]
    not_spam_len = [len(i) for i in not_spam]
    np_nspam = np.array(not_spam_len)
    
    spam = [i for i in spam_data['text'][spam_data['target'] == 1]]
    spam_len = [len(i) for i in spam]
    spam_len = np.array(spam_len)
    
    
    return (np_nspam.mean(), spam_len.mean())

In [15]:
answer_six()

(71.023626943005183, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [16]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [17]:
from sklearn.svm import SVC

def answer_seven():
    tfidf = TfidfVectorizer(min_df = 5)
    vect = tfidf.fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    X_train_upd = add_feature(X_train_vectorized, X_train.str.len())
    X_test_vectorized = vect.transform(X_test)
    X_test_upd = add_feature(X_test_vectorized, X_test.str.len())
    clf = SVC(C = 10000).fit(X_train_upd, y_train)
    pred = clf.predict(X_test_upd)
    auc = roc_auc_score(y_test, pred)
    
    
    return auc

In [18]:
answer_seven()

0.95813668234215565

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [43]:
import re
def answer_eight():
    not_spam = spam_data['text'][spam_data['target'] == 0]
    digits_count = []
    for i in not_spam:
        digit = re.findall(r'\d', i)
        digit_no = len(digit)
        digits_count.append(digit_no)
    digits_count = np.array(digits_count)
    mean = digits_count.mean()
    spam = spam_data['text'][spam_data['target'] == 1]
    digit_count = []
    for i in spam:
        digits = re.findall(r'\d', i)
        digit_no = len(digits)
        digit_count.append(digit_no)
    digit_count = np.array(digit_count)
    mean_1 = digit_count.mean()
    

    
    return (mean, mean_1)

In [44]:
answer_eight()

(0.29927461139896372, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [59]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    tfidf = TfidfVectorizer(min_df = 5, ngram_range = (1,3))
    vect = tfidf.fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    X_train_vectorized = add_feature(X_train_vectorized, X_train.str.len())
    X_train_digits = X_train.str.findall(r'\d')
    X_train_vectorized = add_feature(X_train_vectorized, list(map(len, X_train_digits)))
    X_test_vectorized = vect.transform(X_test)
    X_test_vectorized = add_feature(X_test_vectorized, X_test.str.len())
    X_test_digits = X_test.str.findall(r'\d')
    X_test_vectorized = add_feature(X_test_vectorized, list(map(len, X_test_digits)))
    clf = LogisticRegression(C = 100)
    log = clf.fit(X_train_vectorized, y_train)
    pred = log.predict(X_test_vectorized)
    auc = roc_auc_score(y_test, pred)
    
    
    
    return auc

In [60]:
answer_nine()

0.96533283533945646

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [75]:
import re
def answer_ten():
    non_spam = spam_data['text'][spam_data['target'] == 0]
    ns = []
    for i in non_spam:
        nw = re.findall(r'\W', i)
        ns.append(len(nw))
    ns = np.array(ns)
    mean_ns = ns.mean()
    spam = spam_data['text'][spam_data['target'] == 1]
    s = []
    for i in spam:
        nw1 = re.findall(r'\W', i)
        s.append(len(nw1))
    s = np.array(s)
    mean_s = s.mean()

    
    
    return (mean_ns, mean_s)

In [76]:
answer_ten()

(17.291813471502589, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [118]:
def answer_eleven():
    cv = CountVectorizer(min_df = 5, ngram_range = (2,5), analyzer = 'char_wb')
    vect = cv.fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    X_train_vectorized = add_feature(X_train_vectorized, X_train.str.len())
    X_train_digits = X_train.str.findall(r'\d')
    X_train_vectorized = add_feature(X_train_vectorized, list(map(len, X_train_digits)))
    X_train_nw = X_train.str.findall(r'\W')
    X_train_vectorized = add_feature(X_train_vectorized, list(map(len, X_train_nw)))
    X_test_vectorized = vect.transform(X_test)
    X_test_vectorized = add_feature(X_test_vectorized, X_test.str.len())
    X_test_digits = X_test.str.findall(r'\d')
    X_test_vectorized = add_feature(X_test_vectorized, list(map(len, X_test_digits)))
    X_test_nw = X_test.str.findall(r'\W')
    X_test_vectorized = add_feature(X_test_vectorized, list(map(len, X_test_nw)))
    clf = LogisticRegression(C = 100)
    log = clf.fit(X_train_vectorized, y_train)
    pred = log.predict(X_test_vectorized)
    auc = roc_auc_score(y_test, pred)
    feature_names = np.array(vect.get_feature_names())
    feature_names = np.concatenate((feature_names, np.array(['length_of_doc', 'digit_count', 'non_word_char_count'])))
    smallest = feature_names[clf.coef_[0].argsort()[:10]]
    largest = feature_names[clf.coef_[0].argsort()[:-11:-1]]
    
    
    
    return (auc, smallest.tolist(), largest.tolist())

In [119]:
answer_eleven()

(0.97885931107074342,
 ['. ', '..', '? ', ' i', ' y', ' go', ':)', ' h', 'go', ' m'],
 ['digit_count', 'ne', 'ia', 'co', 'xt', ' ch', 'mob', ' x', 'ww', 'ar'])